In [1]:
#Importing the libraries
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [2]:
news = pd.read_csv('fake_or_real_news.csv')
news.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
X = news['text']
y = news['label']

In [73]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X)

freq_term_matrix = count_vectorizer.transform(X)
tfidf = TfidfTransformer(norm = "l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)


#Splitting the data into train
X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, y, test_size=0.2)


In [74]:
from sklearn.metrics import accuracy_score

from sklearn.linear_model import PassiveAggressiveClassifier

pac=PassiveAggressiveClassifier(max_iter=50)

pac.fit(X_train,y_train)

 #Predict on the test set and calculate accuracy

y_pred=pac.predict(X_test)

score=accuracy_score(y_test,y_pred)

print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 94.16%


In [76]:
#Checking the performance of our model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        FAKE       0.95      0.94      0.94       637
        REAL       0.94      0.95      0.94       630

    accuracy                           0.94      1267
   macro avg       0.94      0.94      0.94      1267
weighted avg       0.94      0.94      0.94      1267

[[596  41]
 [ 33 597]]


In [84]:
#Splitting the data into train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [78]:
#Creating a pipeline that first creates bag of words(after applying stopwords) & then applies Multinomial Naive Bayes model
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                    ('classifier', PassiveAggressiveClassifier(max_iter=50))])

In [79]:
#Training our data
pipeline.fit(X_train, y_train)

#Predicting the label for the test data
pred = pipeline.predict(X_test)

In [80]:
#Checking the performance of our model
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

        FAKE       0.93      0.93      0.93       609
        REAL       0.94      0.93      0.93       658

    accuracy                           0.93      1267
   macro avg       0.93      0.93      0.93      1267
weighted avg       0.93      0.93      0.93      1267

[[569  40]
 [ 46 612]]


In [8]:
#Serialising the file
with open('model1.pickle', 'wb') as handle:
    pickle.dump(pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
#Importing the Libraries
import numpy as np
from flask import Flask, request,render_template
from flask_cors import CORS
import os
import joblib
import joblib
import pickle
import flask
import os
import newspaper
from newspaper import Article
import urllib


In [85]:
X

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [86]:
X_train

1128    Donald Trump's Problems Are Much Deeper Than A...
4007    The ‘Two-Party Racket’ Is Incapable of Dealing...
3357    As predicted by anyone who knows how the FBI w...
354     Australian taxpayers charged over $88 million ...
4080    Kathmandu, Nepal (CNN) The clamor and chaos of...
                              ...                        
3854    We all know by now that the Clinton campaign w...
2870    President Barack Obama‘s budget for next year,...
219     Next Swipe left/right 8 classic football banne...
2715    Diversity Macht Frei October 27, 2016 \nThe Je...
1795    Hillary Clinton expressed regret Saturday for ...
Name: text, Length: 5068, dtype: object

In [87]:
freq_term_matrix

<6335x67659 sparse matrix of type '<class 'numpy.int64'>'
	with 2158282 stored elements in Compressed Sparse Row format>

In [1]:
#Importing the libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

df = pd.read_csv('fake_or_real_news.csv')
df.head()
df = df.fillna(' ')
df.label = df.label.astype(str)
df.label = df.label.str.strip()

X = df['text']
y = df['label']

# count_vectorizer = CountVectorizer()
# count_vectorizer.fit_transform(X)

# freq_term_matrix = count_vectorizer.transform(X)
# tfidf = TfidfTransformer(norm = "l2")
# tfidf.fit(freq_term_matrix)
# tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)


#Splitting the data into train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#Creating a pipeline that first creates bag of words(after applying stopwords) & then applies Multinomial Naive Bayes model
# pipeline = Pipeline([('tfidf', TfidfTransformer(stop_words='english')),
#                     ('classifier', PassiveAggressiveClassifier(max_iter=50))])
pipeline=Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                    ('classifier', PassiveAggressiveClassifier(max_iter=50)),
                    ])

# #Training our data
# pipeline.fit(X_train, y_train)

# #Predicting the label for the test data
# pred = pipeline.predict(X_test)

# pac=PassiveAggressiveClassifier(max_iter=50)

pipeline.fit(X_train,y_train)

 #Predict on the test set and calculate accuracy

pred=pipeline.predict(X_test)


#Checking the performance of our model
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

#Serialising the file
with open('model1.pickle', 'wb') as handle:
    pickle.dump(pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)



              precision    recall  f1-score   support

        FAKE       0.91      0.94      0.93       615
        REAL       0.95      0.92      0.93       652

    accuracy                           0.93      1267
   macro avg       0.93      0.93      0.93      1267
weighted avg       0.93      0.93      0.93      1267

[[581  34]
 [ 55 597]]
